In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

计算delta

In [2]:
def calculate_delta(row):
    S = row['S']
    K = row['K']
    sigma = row['sigma']
    r = row['r']
    T = row['T']
    optype = row['optype']
    positype = row['positype']
    
    d1 = (np.log(S/K) + (r+pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
    
    if optype=='C':
        if positype=='买':
            delta = norm.cdf(d1)
        else:
            delta = -norm.cdf(d1)
    else:
        if positype=='买':
            delta = norm.cdf(d1)-1
        else:
            delta = 1-norm.cdf(d1)
    
    return delta

计算gamma

In [3]:
def calculate_gamma(row):
    S = row['S']
    K = row['K']
    sigma = row['sigma']
    r = row['r']
    T = row['T']
    positype = row['positype']
    
    d1 = (np.log(S/K) + (r+pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
    if positype=='买':
        gamma = np.exp(-pow(d1,2)/2) / (S * sigma * np.sqrt(2*np.pi*T))
    else:
        gamma = -np.exp(-pow(d1,2)/2) / (S * sigma * np.sqrt(2*np.pi*T))
    return gamma

计算theta

In [4]:
def calculate_theta(row):
    S = row['S']
    K = row['K']
    sigma = row['sigma']
    r = row['r']
    T = row['T']
    optype = row['optype']
    positype = row['positype']
    
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    theta_call = -(S*sigma*np.exp(-pow(d1,2)/2)) / (2*np.sqrt(2*np.pi*T)) - r*K*np.exp(-r*T)*norm.cdf(d2)
    
    if optype == 'C':
        if positype=='买':
            theta = theta_call
        else:
            theta = -theta_call
    else:
        if positype=='买':
            theta = theta_call + r*K*np.exp(-r*T)
        else:
            theta = -(theta_call + r*K*np.exp(-r*T))
        
    return theta

构造数据集

In [5]:
# 创建一个示例DataFrame
data = {
    'S': [100, 105, 98],
    'K': [100, 100, 100],
    'sigma': [0.2, 0.15, 0.25],
    'r': [0.05, 0.05, 0.05],
    'T': [1, 0.5, 0.75],
    'optype': ['C', 'P', 'C'],
    'positype': ['买', '卖', '卖']
}

df = pd.DataFrame(data)

In [6]:
df

,S,K,sigma,r,T,optype,positype
0,100,100,0.20,0.05,1.00,C,买
1,105,100,0.15,0.05,0.50,P,卖
2,98,100,0.25,0.05,0.75,C,卖


计算

In [7]:
df['delta'] = df.apply(calculate_delta, axis=1)

In [8]:
df['gamma'] = df.apply(calculate_gamma, axis=1)

In [9]:
df['theta'] = df.apply(calculate_theta, axis=1)

In [10]:
df

,S,K,sigma,r,T,optype,positype,delta,gamma,theta
0,100,100,0.20,0.05,1.00,C,买,0.636831,0.018762,-6.414028
1,105,100,0.15,0.05,0.50,P,卖,0.227009,-0.027065,2.087956
2,98,100,0.25,0.05,0.75,C,卖,-0.574619,-0.018473,7.897575
